# Tratamentos necessários para a criação do One Page Report

* Esse arquivo possui todos os passos realizados para tratar as bases de cadastro e sinistro
* As bases ajustadas/tratadas serão utilizadas para a criação do dashboard one page no Power BI
* O Google Colab desse arquivo pode ser acessado pelo link: https://colab.research.google.com/drive/1Idz9mInvC2IrpDplhHuzcYsRTVXSGhYn?usp=sharing


# 1 - Configuração do ambiente e obtenção dos dados no Google Drive


In [23]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

bd_cadastro = pd.read_excel("/content/drive/MyDrive/Banco de dados/Cadastro_base.xlsx", dtype=str)
bd_sinistro = pd.read_excel("/content/drive/MyDrive/Banco de dados/Sinistros_base.xlsx", dtype=str)
dic_cetipo = pd.read_csv("/content/drive/MyDrive/Banco de dados/compilados cetipos HBIT.csv", sep=";", dtype=str)

# Limpar os nomes das colunas para evitar erros invisíveis
bd_cadastro.columns = bd_cadastro.columns.str.strip()
bd_sinistro.columns = bd_sinistro.columns.str.strip()
dic_cetipo.columns = dic_cetipo.columns.str.strip()

# Limpar espaços a mais nas colunas
for col in bd_cadastro.select_dtypes(include='object').columns:
    bd_cadastro[col] = bd_cadastro[col].astype(str).str.strip().str.replace(r'\s+', ' ', regex=True)

for col in bd_sinistro.select_dtypes(include='object').columns:
    bd_sinistro[col] = bd_sinistro[col].astype(str).str.strip().str.replace(r'\s+', ' ', regex=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#2 - Ajustes na base de cadastro

1. Verificação das informações iniciais da base
*   16.736 linhas
*   10 colunas do tipo string
*   Nenhum dado NULL






In [24]:
print('Verificando tipos de dados:\n')
bd_cadastro.info()

print('\nVisualização das primeiras linhas:\n')
bd_cadastro.head(10)

Verificando tipos de dados:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16736 entries, 0 to 16735
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   MÊS DE REFERÊNCIA       16736 non-null  object
 1   ANO DE REFERÊNCIA       16736 non-null  object
 2   CODIGO DA FAMILIA       16736 non-null  object
 3   CODIGO ÚNICO DA PESSOA  16736 non-null  object
 4   PLANO                   16736 non-null  object
 5   TITULARIDADE            16736 non-null  object
 6   SEXO                    16736 non-null  object
 7   IDADE                   16736 non-null  object
 8   DATA DE NASCIMENTO      16736 non-null  object
 9   FAIXA ETÁRIA            16736 non-null  object
dtypes: object(10)
memory usage: 1.3+ MB

Visualização das primeiras linhas:



,MÊS DE REFERÊNCIA,ANO DE REFERÊNCIA,CODIGO DA FAMILIA,CODIGO ÚNICO DA PESSOA,PLANO,TITULARIDADE,SEXO,IDADE,DATA DE NASCIMENTO,FAIXA ETÁRIA
0,5,2020,314,844,Básico,TITULAR,M,20,16/02/2000,19-23
1,5,2020,46,71,Básico,TITULAR,M,44,21/04/1976,44-48
2,5,2020,123,196,Básico,TITULAR,M,55,20/09/1964,54-58
3,5,2020,114,187,Básico,TITULAR,M,36,21/07/1983,34-38
4,5,2020,52,87,Básico,TITULAR,M,47,15/07/1972,44-48
5,5,2020,55,821,Básico,DEPENDENTE,F,32,19/01/1988,29-33
6,5,2020,118,311,Básico,DEPENDENTE,F,42,02/02/1978,39-43
7,5,2020,326,920,Básico,DEPENDENTE,F,32,14/03/1988,29-33
8,5,2020,21,36,Básico,TITULAR,M,40,24/10/1979,39-43
9,5,2020,263,681,Básico,TITULAR,M,32,25/04/1988,29-33


2. Unificando as colunas de datas
*   Passo realizado para manter 1 padrão de datas






In [25]:
# Criando a coluna 'REFERENCIA' no formato mm/aaaa
bd_cadastro['REFERENCIA'] = '01/' + bd_cadastro['MÊS DE REFERÊNCIA'].str.zfill(2) + '/' + bd_cadastro['ANO DE REFERÊNCIA'].astype(str)

# Removendo as colunas 'MÊS DE REFERÊNCIA' e 'ANO DE REFERÊNCIA'
bd_cadastro = bd_cadastro.drop(['MÊS DE REFERÊNCIA', 'ANO DE REFERÊNCIA'], axis=1)

# Movendo a coluna 'REFERENCIA' para a primeira posição
cols = list(bd_cadastro.columns)
cols.insert(0, cols.pop(cols.index('REFERENCIA')))
bd_cadastro = bd_cadastro.loc[:, cols]

print(bd_cadastro['REFERENCIA'].sort_values().unique())
print('/n')
print(bd_cadastro.head(10))

['01/01/2021' '01/02/2021' '01/03/2021' '01/04/2021' '01/05/2020'
 '01/06/2020' '01/07/2020' '01/08/2020' '01/09/2020' '01/10/2020'
 '01/11/2020' '01/12/2020']
/n
   REFERENCIA CODIGO DA FAMILIA CODIGO ÚNICO DA PESSOA   PLANO TITULARIDADE  \
0  01/05/2020               314                    844  Básico      TITULAR   
1  01/05/2020                46                     71  Básico      TITULAR   
2  01/05/2020               123                    196  Básico      TITULAR   
3  01/05/2020               114                    187  Básico      TITULAR   
4  01/05/2020                52                     87  Básico      TITULAR   
5  01/05/2020                55                    821  Básico   DEPENDENTE   
6  01/05/2020               118                    311  Básico   DEPENDENTE   
7  01/05/2020               326                    920  Básico   DEPENDENTE   
8  01/05/2020                21                     36  Básico      TITULAR   
9  01/05/2020               263                

3. Atualização tipos de dados e remoneação de colunas


In [26]:
#Transformando coluna 'IDADE' para int
bd_cadastro['IDADE'] = bd_cadastro['IDADE'].astype(int)

#Transformando a coluna 'REFERENCIA' para datatime
bd_cadastro['REFERENCIA'] = pd.to_datetime(bd_cadastro['REFERENCIA'], format='%d/%m/%Y', errors='coerce')

#Renomeando colunas para remoção de acentos e espaços
bd_cadastro = bd_cadastro.rename(columns={
    'CODIGO DA FAMILIA': 'CODIGO_FAMILIA',
    'CODIGO ÚNICO DA PESSOA': 'CODIGO_PESSOA',
    'DATA DE NASCIMENTO': 'DATA_NASCIMENTO',
    'FAIXA ETÁRIA': 'FAIXA_ETARIA'
})

print('Verificando tipos de dados:\n')
bd_cadastro.info()

print('\nVisualização das primeiras linhas:\n')
bd_cadastro.head(10)

Verificando tipos de dados:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16736 entries, 0 to 16735
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   REFERENCIA       16736 non-null  datetime64[ns]
 1   CODIGO_FAMILIA   16736 non-null  object        
 2   CODIGO_PESSOA    16736 non-null  object        
 3   PLANO            16736 non-null  object        
 4   TITULARIDADE     16736 non-null  object        
 5   SEXO             16736 non-null  object        
 6   IDADE            16736 non-null  int64         
 7   DATA_NASCIMENTO  16736 non-null  object        
 8   FAIXA_ETARIA     16736 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 1.1+ MB

Visualização das primeiras linhas:



,REFERENCIA,CODIGO_FAMILIA,CODIGO_PESSOA,PLANO,TITULARIDADE,SEXO,IDADE,DATA_NASCIMENTO,FAIXA_ETARIA
0,2020-05-01,314,844,Básico,TITULAR,M,20,16/02/2000,19-23
1,2020-05-01,46,71,Básico,TITULAR,M,44,21/04/1976,44-48
2,2020-05-01,123,196,Básico,TITULAR,M,55,20/09/1964,54-58
3,2020-05-01,114,187,Básico,TITULAR,M,36,21/07/1983,34-38
4,2020-05-01,52,87,Básico,TITULAR,M,47,15/07/1972,44-48
5,2020-05-01,55,821,Básico,DEPENDENTE,F,32,19/01/1988,29-33
6,2020-05-01,118,311,Básico,DEPENDENTE,F,42,02/02/1978,39-43
7,2020-05-01,326,920,Básico,DEPENDENTE,F,32,14/03/1988,29-33
8,2020-05-01,21,36,Básico,TITULAR,M,40,24/10/1979,39-43
9,2020-05-01,263,681,Básico,TITULAR,M,32,25/04/1988,29-33


# 3 - Ajustes na base de sinistros

1. Verificação das informações iniciais da base
*   48.965 linhas
*   **12** colunas do tipo string
*   Nenhum dado NULL


In [27]:
print('Verificando tipos de dados:\n')
bd_sinistro.info()

print('\nVisualização das primeiras linhas:\n')
bd_sinistro.head(10)

Verificando tipos de dados:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48965 entries, 0 to 48964
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   DATA DE REFERENCIA      48965 non-null  object
 1   CODIGO DO EVENTO        48965 non-null  object
 2   FONTE                   48965 non-null  object
 3   PLANO                   48965 non-null  object
 4   TITULARIDADE            48965 non-null  object
 5   CODIGO DA FAMILIA       48965 non-null  object
 6   CODIGO ÚNICO DA PESSOA  48965 non-null  object
 7   DATA DO EVENTO          48965 non-null  object
 8   PRESTADOR               48965 non-null  object
 9   VALOR DO EVENTO         48965 non-null  object
 10  DESCRITOR DO EVENTO     48965 non-null  object
 11  INTERNACAO?             48965 non-null  object
dtypes: object(12)
memory usage: 4.5+ MB

Visualização das primeiras linhas:



,DATA DE REFERENCIA,CODIGO DO EVENTO,FONTE,PLANO,TITULARIDADE,CODIGO DA FAMILIA,CODIGO ÚNICO DA PESSOA,DATA DO EVENTO,PRESTADOR,VALOR DO EVENTO,DESCRITOR DO EVENTO,INTERNACAO?
0,01-05-2020,10101012,REDE,Básico,DEPENDENTE,184,418,13/03/2020,256,64.02,CONSULTA MEDICA EM CONSULTORIO,Não
1,01-05-2020,10101012,REDE,Básico,DEPENDENTE,198,421,17/03/2020,256,65.2,CONSULTA MEDICA EM CONSULTORIO,Não
2,01-05-2020,10101012,REDE,Básico,TITULAR,199,502,13/03/2020,256,65.2,CONSULTA MEDICA EM CONSULTORIO,Não
3,01-05-2020,10101012,REDE,Básico,TITULAR,199,502,18/03/2020,256,65.2,CONSULTA MEDICA EM CONSULTORIO,Não
4,01-05-2020,10101012,REDE,Básico,TITULAR,199,502,08/04/2020,1332,103,CONSULTA MEDICA EM CONSULTORIO,Não
5,01-05-2020,10101012,REDE,Médio,TITULAR,413,1061,05/03/2020,459,64.62,CONSULTA MEDICA EM CONSULTORIO,Não
6,01-05-2020,10101012,REDE,Médio,TITULAR,413,1061,05/03/2020,403,93.83,CONSULTA MEDICA EM CONSULTORIO,Não
7,01-05-2020,10101012,REDE,Básico,DEPENDENTE,219,514,04/03/2020,378,103.01,CONSULTA MEDICA EM CONSULTORIO,Não
8,01-05-2020,10101012,REDE,Básico,DEPENDENTE,226,516,24/03/2020,683,103,CONSULTA MEDICA EM CONSULTORIO,Não
9,01-05-2020,64617270,REDE,Básico,TITULAR,227,552,12/03/2020,295,153.1,PCTE CONSULTA OFTALMOLOGIA TIPO III PCTE COM HM,Não


2. Atualização tipos de dados e remoneação de colunas

In [28]:
#Transformando as colunas 'REFERENCIA' e 'DATA DO EVENTO' para datatime
bd_sinistro['DATA DE REFERENCIA'] = pd.to_datetime(bd_sinistro['DATA DE REFERENCIA'], dayfirst=True, errors='coerce')
bd_sinistro['DATA DO EVENTO'] = pd.to_datetime(bd_sinistro['DATA DO EVENTO'], dayfirst=True, errors='coerce')

#Substituir . para , na coluna de 'VALOR DO EVENTO'
# Garante que a coluna seja float
bd_sinistro['VALOR DO EVENTO'] = pd.to_numeric(bd_sinistro['VALOR DO EVENTO'], errors='coerce')

# Aplica a formatação (float → string com vírgula)
bd_sinistro['VALOR DO EVENTO'] = bd_sinistro['VALOR DO EVENTO'].apply(
    lambda x: f'{x:.2f}'.replace('.', ',') if pd.notnull(x) else ''
)

#Renomeando colunas para remoção de acentos e espaços
bd_sinistro = bd_sinistro.rename(columns={
    'DATA DE REFERENCIA': 'REFERENCIA',
    'CODIGO DO EVENTO': 'CODIGO_EVENTO',
    'CODIGO DA FAMILIA': 'CODIGO_FAMILIA',
    'CODIGO ÚNICO DA PESSOA': 'CODIGO_PESSOA',
    'DATA DO EVENTO': 'DATA_EVENTO',
    'VALOR DO EVENTO': 'VALOR_EVENTO',
    'DESCRITOR DO EVENTO': 'DESCRITOR_EVENTO',
    'INTERNACAO?': 'INTERNACAO'
})

print('Verificando tipos de dados:\n')
bd_sinistro.info()

print('\nVisualização das primeiras linhas:\n')
bd_sinistro.head(10)

Verificando tipos de dados:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48965 entries, 0 to 48964
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   REFERENCIA        48965 non-null  datetime64[ns]
 1   CODIGO_EVENTO     48965 non-null  object        
 2   FONTE             48965 non-null  object        
 3   PLANO             48965 non-null  object        
 4   TITULARIDADE      48965 non-null  object        
 5   CODIGO_FAMILIA    48965 non-null  object        
 6   CODIGO_PESSOA     48965 non-null  object        
 7   DATA_EVENTO       48965 non-null  datetime64[ns]
 8   PRESTADOR         48965 non-null  object        
 9   VALOR_EVENTO      48965 non-null  object        
 10  DESCRITOR_EVENTO  48965 non-null  object        
 11  INTERNACAO        48965 non-null  object        
dtypes: datetime64[ns](2), object(10)
memory usage: 4.5+ MB

Visualização das primeiras linhas:



,REFERENCIA,CODIGO_EVENTO,FONTE,PLANO,TITULARIDADE,CODIGO_FAMILIA,CODIGO_PESSOA,DATA_EVENTO,PRESTADOR,VALOR_EVENTO,DESCRITOR_EVENTO,INTERNACAO
0,2020-05-01,10101012,REDE,Básico,DEPENDENTE,184,418,2020-03-13,256,"64,02",CONSULTA MEDICA EM CONSULTORIO,Não
1,2020-05-01,10101012,REDE,Básico,DEPENDENTE,198,421,2020-03-17,256,"65,20",CONSULTA MEDICA EM CONSULTORIO,Não
2,2020-05-01,10101012,REDE,Básico,TITULAR,199,502,2020-03-13,256,"65,20",CONSULTA MEDICA EM CONSULTORIO,Não
3,2020-05-01,10101012,REDE,Básico,TITULAR,199,502,2020-03-18,256,"65,20",CONSULTA MEDICA EM CONSULTORIO,Não
4,2020-05-01,10101012,REDE,Básico,TITULAR,199,502,2020-04-08,1332,"103,00",CONSULTA MEDICA EM CONSULTORIO,Não
5,2020-05-01,10101012,REDE,Médio,TITULAR,413,1061,2020-03-05,459,"64,62",CONSULTA MEDICA EM CONSULTORIO,Não
6,2020-05-01,10101012,REDE,Médio,TITULAR,413,1061,2020-03-05,403,"93,83",CONSULTA MEDICA EM CONSULTORIO,Não
7,2020-05-01,10101012,REDE,Básico,DEPENDENTE,219,514,2020-03-04,378,"103,01",CONSULTA MEDICA EM CONSULTORIO,Não
8,2020-05-01,10101012,REDE,Básico,DEPENDENTE,226,516,2020-03-24,683,"103,00",CONSULTA MEDICA EM CONSULTORIO,Não
9,2020-05-01,64617270,REDE,Básico,TITULAR,227,552,2020-03-12,295,"153,10",PCTE CONSULTA OFTALMOLOGIA TIPO III PCTE COM HM,Não


# 4 - Classificação Cetipos - Hbit

* **Passo necessário para possibilitar análises por tipo de evento. Ele tem o proposito de realizar a classificação dos eventos da base de sinistro com cruzamento com o arquivo classificação Hbit.csv**
<br>

* Esse arquivo foi criado no Excel a partir dos dados compilados de códigos de eventos e suas classificação Hbit.
* O compilado foi montado a partir dos dicionários de cetipos das operadoras Seguros Unimed, CNU, Intermédica, Unimed Vitória e Unimed Santos (Essas operadoras foram escolhidos porque possuiam códigos de eventos no padrão TUSS, igual a base de sinistro).
<br>

* 109 eventos não puderam ser classficiados com o compilado. Por ser uma quantidade pequena, decidi realizar a classificação manual e atualizei o compilado com essas informações.

In [29]:
# Renomeia a coluna do dic_cetipo para coincidir com o nome da coluna no sinistro
dic_cetipo.rename(columns={'PROCEDIMENTO': 'CODIGO_EVENTO'}, inplace=True)

dic_cetipo.head(10)

,CODIGO_EVENTO,CETIPO NOVO,SEGURADORA
0,0,outros,Seguros_Unimed
1,1,outros,Seguros_Unimed
2,00000001,outros,Seguros_Unimed
3,00000003,outros,Seguros_Unimed
4,00000006,outros,Seguros_Unimed
5,00000007,outros,Seguros_Unimed
6,0000000008,outros,Seguros_Unimed
7,00000008,outros,Seguros_Unimed
8,10,outros,Seguros_Unimed
9,0000000010,outros,Seguros_Unimed


In [30]:
# Merge com dic_cetipo (sem filtrar ainda)
bd_sinistro = bd_sinistro.merge(
    dic_cetipo[['CODIGO_EVENTO', 'CETIPO NOVO']],
    on='CODIGO_EVENTO',
    how='left'
)

In [31]:
# Cria uma nova coluna 'CLASSIFICACAO_HBIT' e realiza a classificação dos eventos que não são classificados como 'INTERNACAO'
bd_sinistro['CLASSIFICACAO_HBIT'] = np.where(
    bd_sinistro['INTERNACAO'].str.upper().str.strip() == 'SIM',
    'INTERNACAO',
    bd_sinistro['CETIPO NOVO']
)

In [32]:
#Remove a coluna auxiliar do merge
bd_sinistro.drop(columns=['CETIPO NOVO'], inplace=True)

bd_sinistro.info()
bd_sinistro.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48965 entries, 0 to 48964
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   REFERENCIA          48965 non-null  datetime64[ns]
 1   CODIGO_EVENTO       48965 non-null  object        
 2   FONTE               48965 non-null  object        
 3   PLANO               48965 non-null  object        
 4   TITULARIDADE        48965 non-null  object        
 5   CODIGO_FAMILIA      48965 non-null  object        
 6   CODIGO_PESSOA       48965 non-null  object        
 7   DATA_EVENTO         48965 non-null  datetime64[ns]
 8   PRESTADOR           48965 non-null  object        
 9   VALOR_EVENTO        48965 non-null  object        
 10  DESCRITOR_EVENTO    48965 non-null  object        
 11  INTERNACAO          48965 non-null  object        
 12  CLASSIFICACAO_HBIT  48965 non-null  object        
dtypes: datetime64[ns](2), object(11)
memory usage:

,REFERENCIA,CODIGO_EVENTO,FONTE,PLANO,TITULARIDADE,CODIGO_FAMILIA,CODIGO_PESSOA,DATA_EVENTO,PRESTADOR,VALOR_EVENTO,DESCRITOR_EVENTO,INTERNACAO,CLASSIFICACAO_HBIT
0,2020-05-01,10101012,REDE,Básico,DEPENDENTE,184,418,2020-03-13,256,"64,02",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
1,2020-05-01,10101012,REDE,Básico,DEPENDENTE,198,421,2020-03-17,256,"65,20",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
2,2020-05-01,10101012,REDE,Básico,TITULAR,199,502,2020-03-13,256,"65,20",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
3,2020-05-01,10101012,REDE,Básico,TITULAR,199,502,2020-03-18,256,"65,20",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
4,2020-05-01,10101012,REDE,Básico,TITULAR,199,502,2020-04-08,1332,"103,00",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
5,2020-05-01,10101012,REDE,Médio,TITULAR,413,1061,2020-03-05,459,"64,62",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
6,2020-05-01,10101012,REDE,Médio,TITULAR,413,1061,2020-03-05,403,"93,83",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
7,2020-05-01,10101012,REDE,Básico,DEPENDENTE,219,514,2020-03-04,378,"103,01",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
8,2020-05-01,10101012,REDE,Básico,DEPENDENTE,226,516,2020-03-24,683,"103,00",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
9,2020-05-01,64617270,REDE,Básico,TITULAR,227,552,2020-03-12,295,"153,10",PCTE CONSULTA OFTALMOLOGIA TIPO III PCTE COM HM,Não,consulta_geral


In [33]:
# Filtrar apenas os eventos sem classificação
nao_classificados = bd_sinistro[bd_sinistro['CLASSIFICACAO_HBIT'].isnull()]

nao_classificados_unicos = nao_classificados[['CODIGO_EVENTO', 'DESCRITOR_EVENTO']].drop_duplicates()

print(f'Total de CÓDIGOS DE EVENTO únicos sem classificação: {len(nao_classificados_unicos)}')

Total de CÓDIGOS DE EVENTO únicos sem classificação: 0


In [34]:
'''
  #Na primeira execução, 109 eventos ainda ficaram sem classificação (Mes utilizando o compilado de classificados do drive)
  #Separei esses eventos para fazer a classificação manual

  # Define o caminho completo para salvar o arquivo
  caminho_nao_classificados_unicos = '/content/drive/MyDrive/Banco de dados/Cetipos Classificados Manualmente/Cetipos_Para_Classificar_Manualmente.csv'

  # Salva o DataFrame como CSV (ex: separador ; e encoding compatível com Excel)
  nao_classificados_unicos.to_csv(caminho_nao_classificados_unicos, sep=';', index=False, encoding='utf-8')

  print(f'Arquivo salvo no drive')
'''

"\n  #Na primeira execução, 109 eventos ainda ficaram sem classificação (Mes utilizando o compilado de classificados do drive)\n  #Separei esses eventos para fazer a classificação manual\n\n  # Define o caminho completo para salvar o arquivo\n  caminho_nao_classificados_unicos = '/content/drive/MyDrive/Banco de dados/Cetipos Classificados Manualmente/Cetipos_Para_Classificar_Manualmente.csv'\n\n  # Salva o DataFrame como CSV (ex: separador ; e encoding compatível com Excel)\n  nao_classificados_unicos.to_csv(caminho_nao_classificados_unicos, sep=';', index=False, encoding='utf-8')\n\n  print(f'Arquivo salvo no drive')\n"

In [35]:
#Padroniza a escrita de Internação e redução das consultas em PS
bd_sinistro['CLASSIFICACAO_HBIT'] = bd_sinistro['CLASSIFICACAO_HBIT'].replace('INTERNACAO', 'internação')
bd_sinistro['CLASSIFICACAO_HBIT'] = bd_sinistro['CLASSIFICACAO_HBIT'].replace('internacao', 'internação')
bd_sinistro['CLASSIFICACAO_HBIT'] = bd_sinistro['CLASSIFICACAO_HBIT'].replace('consulta_pronto_socorro', 'consulta_PS')

bd_sinistro.head(10)

,REFERENCIA,CODIGO_EVENTO,FONTE,PLANO,TITULARIDADE,CODIGO_FAMILIA,CODIGO_PESSOA,DATA_EVENTO,PRESTADOR,VALOR_EVENTO,DESCRITOR_EVENTO,INTERNACAO,CLASSIFICACAO_HBIT
0,2020-05-01,10101012,REDE,Básico,DEPENDENTE,184,418,2020-03-13,256,"64,02",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
1,2020-05-01,10101012,REDE,Básico,DEPENDENTE,198,421,2020-03-17,256,"65,20",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
2,2020-05-01,10101012,REDE,Básico,TITULAR,199,502,2020-03-13,256,"65,20",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
3,2020-05-01,10101012,REDE,Básico,TITULAR,199,502,2020-03-18,256,"65,20",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
4,2020-05-01,10101012,REDE,Básico,TITULAR,199,502,2020-04-08,1332,"103,00",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
5,2020-05-01,10101012,REDE,Médio,TITULAR,413,1061,2020-03-05,459,"64,62",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
6,2020-05-01,10101012,REDE,Médio,TITULAR,413,1061,2020-03-05,403,"93,83",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
7,2020-05-01,10101012,REDE,Básico,DEPENDENTE,219,514,2020-03-04,378,"103,01",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
8,2020-05-01,10101012,REDE,Básico,DEPENDENTE,226,516,2020-03-24,683,"103,00",CONSULTA MEDICA EM CONSULTORIO,Não,consulta_geral
9,2020-05-01,64617270,REDE,Básico,TITULAR,227,552,2020-03-12,295,"153,10",PCTE CONSULTA OFTALMOLOGIA TIPO III PCTE COM HM,Não,consulta_geral


# 5 - Exportar bases de dados ajustadas com codificação UTF-8

In [36]:
# Define o caminho completo para salvar o arquivo
caminho_arquivo_PC = '/content/drive/MyDrive/Banco de dados/Ajustado/Cadastro_base_ajustado.csv'
caminho_arquivo_SN = '/content/drive/MyDrive/Banco de dados/Ajustado/Sinistros_base_ajustado.csv'

# Salva o DataFrame como CSV (ex: separador ; e encoding compatível com Excel)
bd_cadastro.to_csv(caminho_arquivo_PC, sep=';',date_format='%d/%m/%Y', index=False, encoding='utf-8')
bd_sinistro.to_csv(caminho_arquivo_SN, sep=';', date_format='%d/%m/%Y', index=False, encoding='utf-8')

print(f'Arquivos salvos no drive')

Arquivos salvos no drive
